In [9]:
import chromadb
from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv
from chromadb.utils import embedding_functions
import os
from chromadb.config import Settings

In [10]:
load_dotenv()
CHROMA_CLIENT = chromadb.Client(Settings(
        chroma_db_impl="duckdb+parquet", persist_directory="../chromadb"))
CHROMA_CLIENT.persist()
collection = CHROMA_CLIENT.get_or_create_collection(name="teli-ai")

Using embedded DuckDB with persistence: data will be stored in: ../chromadb
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [12]:
persist_directory = "../chromadb"
collection_name = "teli-ai"
chroma_settings = Settings(
    chroma_db_impl="duckdb+parquet", persist_directory=persist_directory)
CHROMA_CLIENT = chromadb.Client(chroma_settings)
CHROMA_CLIENT.persist()
collection = CHROMA_CLIENT.get_or_create_collection(name=collection_name)
print(collection.peek(1))

Using embedded DuckDB with persistence: data will be stored in: ../chromadb
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


{'ids': ['01fc5148-5d8a-494c-96a0-a70047bde4a1'], 'embeddings': [[-0.1089685782790184, 0.00016724829038139433, 0.02578457072377205, 0.011033688671886921, 0.01992904767394066, -0.051289450377225876, 0.16653332114219666, 0.019782867282629013, 0.005987880751490593, -0.0781824067234993, -0.0011451853206381202, 0.0015390054322779179, -0.03971976786851883, -0.04246683418750763, -0.015420492738485336, 0.022976821288466454, 0.018136538565158844, -0.003676749998703599, -0.0615522563457489, 0.012131087481975555, 0.016443241387605667, 0.04129496216773987, -0.07216466218233109, -0.02821928821504116, -0.023291664198040962, -0.03130220249295235, 0.03417819365859032, 0.04771081358194351, -0.0628955140709877, -0.0332547128200531, -0.032153647392988205, 0.0934462770819664, 0.07621275633573532, 0.004378495272248983, 0.017841441556811333, 0.03956383466720581, -0.06028948351740837, -0.046230100095272064, -0.008978717029094696, 0.00375845143571496, 0.029894458130002022, -0.05491563305258751, -0.08103304356

In [ ]:
collection.peek()

In [7]:
response = collection.query(query_texts=["Customer: Hello"],n_results=2)

In [8]:
suggestions = ""
for doc in response["documents"]:
    for line in doc:
        suggestions += line + "\n\n"

print(suggestions)


Customer: Hello
AI: Hi is this the property owner of

Customer: Not interested 
AI: Okay, have a nice day 




In [26]:
print(response)

{'ids': [['32598dbf-3d29-4ec1-a60d-e76e441793a4']], 'embeddings': None, 'documents': [['Customer: Hello\nAI: Hi is this the property owner of']], 'metadatas': [[None]], 'distances': [[0.8536350727081299]]}


In [ ]:
from transformers import GPT2Tokenizer

def count_gpt_tokens(passage):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokens = tokenizer.encode(passage, add_special_tokens=True)
    token_count = len(tokens)
    return token_count


In [ ]:
prompt = "Act as a telemarketer. Your name is Amelio from Pure Investment. Follow this process:\nYou initiated the call to customer. First, you need to ask customer question to make sure you are talking to the right customer. Then, you need to ask if he/she is living on the right adress.If the information is correct, ask if he/she is interested in selling their current property for cash.\nYou are talking to {customer_name}, living in {customer_address}.\nOnly when the customer is interested in selling their property, ask them for the property condition. Then, ask if they would sell it for {price}.\nOtherwise, if customer is not interested or it is the wrong cusomter, politely end the conversation and hang up.\nYou only address the customer's previous question and avoid providing extra information or generating unrelated sequences.\n\nSample conversation flow:\n{sample_conversation}\n\nCurrent conversation:\nYou : [initiate call]\n".format(customer_name="Vinny", customer_address="Vinny", price="123123",sample_conversation="sample_conversation")
token_count = count_gpt_tokens(prompt)
print("Number of GPT tokens:", token_count)